# Model Pruning

In [1]:
# from torchsummary import summary
import albumentations as A
# import mlflow
# import torch.nn as nn
# import torch.optim as optim
# import tqdm
from albumentations.pytorch import ToTensorV2
# from torch.optim.lr_scheduler import PolynomialLR
# import cv2
# from DiceBCELoss import DiceBCELoss
# from model import UNetPlusPlus, ResNeXtBottleneck, ResBlock
from utils import *
import time

In [2]:
device = "cuda" if torch.cuda.is_available() else "cpu"
batch_size=4

In [3]:
train_transform = A.Compose([
    A.VerticalFlip(p=0.5),
    A.HorizontalFlip(p=0.5),
    A.ShiftScaleRotate(
        shift_limit=0.3,
        scale_limit=0.5,
        rotate_limit=30, 
        border_mode=cv2.BORDER_REFLECT,
        p=0.5
    ),
    A.Affine(shear=5, p=0.5),
    A.Perspective(scale=(0.001, 0.001), p=0.5),
     ToTensorV2()
])

val_transform = A.Compose([
    ToTensorV2()  # This ensures the output is in the format PyTorch expects
])

_, stenosis_test, _ = get_loaders('dataset/stenosis/train', 'dataset/stenosis/test',
                                                          'dataset/stenosis/val', batch_size, train_transform,
                                                          val_transform, val_transform)

/work/m43huang/.local/lib/python3.11/site-packages/albumentations/core/validation.py:87: UserWarning: ShiftScaleRotate is a special case of Affine transform. Please use Affine transform instead.
  original_init(self, **validated_kwargs)


In [4]:
model = torch.load("./checkpoint/sten_unet++_deep_supervision/best.pth")

In [12]:
for i in range(1,5):
    avg_f1, avg_iou, avg_dsc, runtime = track_metric(stenosis_test, model, device=device, post_processing=False, l=i, benchmark=True)
    print(f"UNet++ L({i}) without post-processing: {avg_f1}, {avg_iou}, {avg_dsc}")
    avg_f1, avg_iou, avg_dsc = track_metric(stenosis_test, model, device=device, post_processing=600, l=i)
    print(f"UNet++ L({i}) with post-processing: {avg_f1}, {avg_iou}, {avg_dsc}")
    print(f"{'-'*15} Runtime: {runtime*1e-9:.4f}s")

UNet++ L(1) without post-processing: 0.2420257642543379, 0.14060769285971803, 0.24202576425732744
UNet++ L(1) with post-processing: 0.21381404021530911, 0.12442527496594293, 0.21381404021979308
--------------- Runtime: 0.1252s
UNet++ L(2) without post-processing: 0.3868796768114903, 0.2444783650028748, 0.386879676814891
UNet++ L(2) with post-processing: 0.3106268932970733, 0.19460390212308376, 0.31062689330296855
--------------- Runtime: 0.1919s
UNet++ L(3) without post-processing: 0.5176156703956384, 0.357965332905696, 0.5176156703982292
UNet++ L(3) with post-processing: 0.5199452014371198, 0.36298803441384125, 0.5199452014399903
--------------- Runtime: 0.2470s
UNet++ L(4) without post-processing: 0.5757653585261783, 0.413384362934059, 0.575765358528384
UNet++ L(4) with post-processing: 0.5776053783651814, 0.4168090191766692, 0.5776053783675202
--------------- Runtime: 0.2847s
